In [2]:
import ecole
import shutil
import os
import sys
import time
import numpy as np
from tqdm import trange
import gurobipy as gb
import pyscipopt as pyscip
from gurobipy import GRB
import pickle as pkl

from dump_data import *

scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "presolving/maxrounds": 0,
#                    "constraints/linear/singletonstuffing": False,
#                    "constraints/linear/rangedrowartcons": False,
#                    "constraints/and/maxprerounds" :0,
#                    "constraints/and/presolusehashing": False,
#                    "constraints/benders/maxprerounds": 0,
#                    "constraints/benderslp/iterlimit": 0,
#                    "reading/dynamicrows": False,
#                    "separator/oddcycle/cutthreshold":0,
                  }

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


# Generate Problem and collect features

In [3]:
if os.path.exists("DataSet/"):
    shutil.rmtree("DataSet/")
os.mkdir("DataSet/")

env = ecole.environment.Branching(\
            observation_function = ecole.observation.NodeBipartite(),scip_params = scip_parameters)

for row in [100,200,300,400,500]:
    for coef_col in [1,1.5,2]: 
        for d in [0.1,0.15,0.2]:
            col = int(coef_col * row)
            setCover = ecole.instance.SetCoverGenerator(n_rows = row, n_cols = col,density = d) 
            print("Generate with Row:%d,Col:%d,Density:%f" % (row,col,d))
            for n in trange(1,101):
                done = False
                while(not done):
                    try:
                        problme_name = \
                            "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
                        instance = next(setCover)
                        #save problm lp
                        problme_name = \
                            "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
                        os.mkdir("DataSet/"+problme_name)
                        instance.write_problem("DataSet/"+problme_name+"/problem.lp")
                        #save features
                        obs, _, _, _, _ = env.reset(instance)
                        dumpRowFeatures("DataSet/"+problme_name+"/constraints_features.json",obs.row_features)
                        dumpVariableFeatures("DataSet/"+problme_name+"/variables_features.json",obs.variable_features)
                        dumpEdgeFeatures("DataSet/"+problme_name+"/edges_features.pkl",obs.edge_features)
                        done = True             
                    except Exception as ex:
#                         print("Erreur:%s"%ex)
                        done = False
                        shutil.rmtree("DataSet/"+problme_name)
            

Generate with Row:100,Col:100,Density:0.100000


100%|█████████████████████████████████████████| 100/100 [01:03<00:00,  1.58it/s]


Generate with Row:100,Col:100,Density:0.150000


100%|█████████████████████████████████████████| 100/100 [01:06<00:00,  1.50it/s]


Generate with Row:100,Col:100,Density:0.200000


...

100%|█████████████████████████████████████████| 100/100 [11:04<00:00,  6.65s/it]


Generate with Row:500,Col:1000,Density:0.150000


100%|█████████████████████████████████████████| 100/100 [07:21<00:00,  4.42s/it]


Generate with Row:500,Col:1000,Density:0.200000


100%|█████████████████████████████████████████| 100/100 [06:58<00:00,  4.18s/it]


# Collect labels

In [5]:
if os.path.exists("DataSet/label/"):
    shutil.rmtree("DataSet/label/")
os.mkdir("DataSet/label/")
gapList = []
for row in [100,200,300,400,500]:
    for coef_col in [1,1.5,2]: 
        for d in [0.1,0.15,0.2]:
            col = int(coef_col * row)
            print("Solve with Row:%d,Col:%d,Density:%f" % (row,col,d))
            for n in trange(1,101):
#                 #save label
                with HiddenPrints():
                    problme_name = \
                            "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
#                     solver = gb.read("problem/"+problme_name+".lp")
#                     solver.optimize()
#                     dumpSolution_Gurobi("DataSet/label/"+problme_name+"_label.json",solver)
                    solver = ecole.scip.Model.from_file("DataSet/"+problme_name+"/problem.lp")
                    aspyscip = solver.as_pyscipopt()
                    aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
                    aspyscip.optimize()
                    gapList.append(aspyscip.getGap())
                    dumpSolution_Ecole("DataSet/"+problme_name+"/label.json",aspyscip)
                    
gap = np.array(gapList)
print(np.mean(gap))
print(np.where(gap != 0))

Solve with Row:100,Col:100,Density:0.100000


100%|█████████████████████████████████████████| 100/100 [00:21<00:00,  4.63it/s]


Solve with Row:100,Col:100,Density:0.150000


100%|█████████████████████████████████████████| 100/100 [00:22<00:00,  4.39it/s]


Solve with Row:100,Col:100,Density:0.200000


...

Solve with Row:500,Col:1000,Density:0.150000


100%|█████████████████████████████████████████| 100/100 [09:47<00:00,  5.87s/it]


Solve with Row:500,Col:1000,Density:0.200000


100%|█████████████████████████████████████████| 100/100 [08:10<00:00,  4.90s/it]

0.0
(array([], dtype=int64),)


In [ ]:
#scip
with HiddenPrints():
    model_scip = pyscip.scip.Model()
    model_scip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
    model_scip.readProblem(filename="debugFile/set_cover_test.lp",extension = "lp")
    # model_scip.readProblem(filename="debugFile/debug_diffc.lp",extension = "lp")

model_scip.optimize()
# time.sleep(1)
print("\n",model_scip.getBestSol())

# print("Ecole*********************")
# # with HiddenPrints():
# solver = ecole.scip.Model.from_file("debugFile/debug.lp")
# aspyscip = solver.as_pyscipopt()
# aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
# aspyscip.optimize()
# gap = aspyscip.getGap()
# sol = aspyscip.getBestSol()
# print(sol,gap)

# #gurobi
# print("Gurobi*********************")
# with HiddenPrints():
#     model = gb.read("debugFile/debug.lp")
#     model.optimize()
# for var in model.getVars():
#     print(f"{var.varName}: {round(var.X, 3)}")



In [ ]:
setCover = ecole.instance.SetCoverGenerator\
            (n_rows=300, n_cols=300,density=0.1)
env = ecole.environment.Branching(observation_function = ecole.observation.NodeBipartite(),\
                                scip_params = scip_parameters)
instance = next(setCover)
print(instance)
problme_name = "set_cover_test"
instance.write_problem("debugFile/"+problme_name+".lp")

obs, action_set, _, _, _ = env.reset(instance)
print(obs.row_features.shape)
print(obs.edge_features.shape)
print(obs.variable_features.shape)

# setCover = ecole.instance.SetCoverGenerator\
#             (n_rows=300, n_cols=300,density=0.1)
# env = ecole.environment.Branching(observation_function = ecole.observation.NodeBipartite(),\
#                                 scip_params = scip_parameters)
# instance = next(setCover)
# print(instance)
# problme_name = "set_cover_test"
# instance.write_problem("debugFile/"+problme_name+".lp")


# dynamic = ecole.dynamics.BranchingDynamics()
# dynamic.reset_dynamics(instance)
# obsteur = ecole.observation.NodeBipartite()
# obsteur.before_reset(instance)
# obs= obsteur.extract(instance,False)
# print(obs.row_features.shape)
# dumpRowFeatures(problme_name+"_row_features.json",obs.row_features)

In [ ]:
env = ecole.environment.Branching(observation_function = ecole.observation.NodeBipartite(),\
                                scip_params = scip_parameters)
instance = ecole.scip.Model.from_file("debugFile/test_feature.lp")
print(instance)

obs, action_set, _, _, _ = env.reset(instance)
print(obs.row_features.shape)
print(obs.edge_features.shape)
print(obs.variable_features.shape)
print(obs.row_features[0])
print(obs.variable_features[0])

In [ ]:
gap_test = gapList

In [ ]:
for row in [100,150,200]:
    for coef_col in [1,1.5,2]: 
        for d in [0.1,0.15,0.2]:
            col = int(coef_col * row)
            print("Solve with Row:%d,Col:%d,Density:%f" % (row,col,d))
            for n in range(1,2):
#                 #save label
                problme_name = \
                        "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
#                     solver = gb.read("problem/"+problme_name+".lp")
#                     solver.optimize()
#                     dumpSolution_Gurobi("DataSet/label/"+problme_name+"_label.json",solver)
#                 solver = ecole.scip.Model.from_file("DataSet/problem/"+problme_name+".lp")
#                 aspyscip = solver.as_pyscipopt()
#                 aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
#                 aspyscip.optimize()
                model_scip = pyscip.scip.Model()
                model_scip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
                model_scip.readProblem(filename="DataSet/problem/"+problme_name+".lp")
                model_scip.optimize()